In [1]:
!pip install selenium
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [5]:
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize
from collections import Counter
from scipy.stats import zipf
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
from numpy.linalg import norm
from math import log
import numpy as np
import seaborn as sns
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment  import  SentimentIntensityAnalyzer
stopwords = stopwords.words('english')

In [6]:
#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome('C:/Users/HP/Downloads/chromedriver_win32/chromedriver',options=chrome_options)
wd.get('https://www.thetrumparchive.com/')


C:\Users\HP\AppData\Local\Temp\ipykernel_39428\2837482839.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome('C:/Users/HP/Downloads/chromedriver_win32/chromedriver',options=chrome_options)


In [215]:

def extractDetails(tweetElements,tweetDetailsDf):
    print('Extracting details for {} tweets'.format(len(tweetElements)))
    for index,tweet in enumerate(tweetElements):
        wasDeleted=False
        date = tweet.find_element(by=By.CLASS_NAME,value='date___1q2oK')
        date_time = re.sub(' EST$','',re.sub('(\d+)th|rd|st|nd','\g<1>',date.text))
        dateObject = datetime.strptime(date_time,'%b %d %Y - %I:%M:%S %p')
        details = tweet.find_element(by=By.CLASS_NAME,value='icons___3WEom')
        retweets = details.find_elements(by=By.TAG_NAME,value='span')[0].text
        likes = details.find_elements(by=By.TAG_NAME,value='span')[1].text
        delete = details.find_elements(by=By.TAG_NAME,value='span')[2].text
        if(delete=='Deleted'):
            wasDeleted=True
        tweet = tweet.find_element(by=By.CLASS_NAME,value='text___k4s4q.plain___1AdbJ').text
        print("Row index {}".format(index))
        tweetDetailsDf.loc[len(tweetDetailsDf)] = [dateObject,retweets,likes,tweet,wasDeleted]
    return tweetDetailsDf

    

In [282]:
import re
import time
from datetime import date
from dateutil.relativedelta import relativedelta

def crawler(dateFrom,dateTo,tweetDetailsDf):
    while dateFrom>=date(2009,1,1):
        mainUrl = 'https://www.thetrumparchive.com/?dates=%5B%22{}%22%2C%22{}%22%5D'.format(dateFrom,dateTo)
        print("Extracting tweets from {}".format(mainUrl))
        wd.get(mainUrl)
        time.sleep(0.5)
        end = False
        tableElement = wd.find_element(by=By.CLASS_NAME,value='list___2uPLh')
        temp = tableElement.find_element(by=By.CLASS_NAME,value='css-145p3ou.e1hwwpu60')
        tweetNumbersText= temp.find_elements(by=By.TAG_NAME,value='span')
        tweetNumber = int(tweetNumbersText[0].text.replace(',',''))
        print("Tweets between {} and {} are {}".format(dateFrom,dateTo,tweetNumber))
        while end==False:
            tweetElements = tableElement.find_elements(by=By.CLASS_NAME,value='tweet___2xXtA.ttaTweet')
            wd.execute_script("return arguments[0].scrollIntoView();", tweetElements[-1])
            time.sleep(0.8)
            print(len(tweetElements))
            if(len(tweetElements)==tweetNumber):
                end=True
        tweetDetailsDf = extractDetails(tweetElements,tweetDetailsDf)
        dateTo = dateFrom-relativedelta(days=1)
        dateFrom = dateFrom-relativedelta(months=4)
        print("New dateFrom : {}".format(dateFrom))
        print("New dateTo : {}".format(dateTo))
    return tweetDetailsDf

In [283]:
tweetDetailsDf = pd.DataFrame(columns=['Date','Retweets','Likes','Tweet','WasDeleted'])
resultsDf = crawler(date(2009,1,1),date(2011,1,1),tweetDetailsDf)

Extracting tweets from https://www.thetrumparchive.com/?dates=%5B%222009-01-01%22%2C%222011-01-01%22%5D
Tweets between 2009-01-01 and 2011-01-01 are 198
25
50
75
100
125
150
175
198
Extracting details for 198 tweets
Row index 0
Row index 1
Row index 2
Row index 3
Row index 4
Row index 5
Row index 6
Row index 7
Row index 8
Row index 9
Row index 10
Row index 11
Row index 12
Row index 13
Row index 14
Row index 15
Row index 16
Row index 17
Row index 18
Row index 19
Row index 20
Row index 21
Row index 22
Row index 23
Row index 24
Row index 25
Row index 26
Row index 27
Row index 28
Row index 29
Row index 30
Row index 31
Row index 32
Row index 33
Row index 34
Row index 35
Row index 36
Row index 37
Row index 38
Row index 39
Row index 40
Row index 41
Row index 42
Row index 43
Row index 44
Row index 45
Row index 46
Row index 47
Row index 48
Row index 49
Row index 50
Row index 51
Row index 52
Row index 53
Row index 54
Row index 55
Row index 56
Row index 57
Row index 58
Row index 59
Row index 60
R

In [284]:
repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9'}

resultsDf['Retweets'] = resultsDf['Retweets'].replace(repl_dict, regex=True).map(pd.eval).astype(int)
resultsDf['Likes'] = resultsDf['Likes'].replace(repl_dict, regex=True).map(pd.eval).astype(int)

In [285]:
resultsDf.to_csv('TrumpTweets_14.csv',index=False)

In [233]:
resultsDf

,Date,Retweets,Likes,Tweet,WasDeleted
0,2020-08-30 23:03:34,49000,215000,Just had a wonderful conversation with my frie...,False
1,2020-08-30 22:36:13,33000,144000,When is Slow Joe Biden going to criticize the ...,False
2,2020-08-30 22:25:50,15000,72000,....His problem is interesting. He must always...,False
3,2020-08-30 22:25:49,26000,119000,Joe Biden is coming out of the basement earlie...,False
4,2020-08-30 21:18:30,59000,0,RT @realDonaldTrump: LAW & ORDER!!!,False
...,...,...,...,...,...
4267,2020-04-30 07:59:20,7000,37000,Really? Have you looked at the numbers lately!...,False
4268,2020-04-30 07:56:44,18000,109000,"FAKE POLLING, just like 2016 (but worse)!",False
4269,2020-04-30 07:47:47,35000,153000,"What happened to General Michael Flynn, a war ...",False
4270,2020-04-30 07:47:04,27000,94000,"Does anybody really believe that Roger Stone, ...",False


In [129]:
import re
from datetime import date
from dateutil.relativedelta import relativedelta
tweetDetailsDf = pd.DataFrame(columns=['Date','Retweets','Likes','Tweet','WasDeleted'])
datefrom= date(2020,1,10)
dateto=date(2020,7,10)
url = 'https://www.thetrumparchive.com/?dates=%5B%22{}%22%2C%22{}%22%5D'.format(datefrom-relativedelta(months=6),dateto-relativedelta(months=6))
url

'https://www.thetrumparchive.com/?dates=%5B%222019-07-02%22%2C%222020-01-01%22%5D'

In [114]:
repl_dict = {'[kK]': '*1e3', '[mM]': '*1e6', '[bB]': '*1e9'}

tweetDetailsDf['Retweets'] = tweetDetailsDf['Retweets'].replace(repl_dict, regex=True).map(pd.eval).astype(int)
tweetDetailsDf['Likes'] = tweetDetailsDf['Likes'].replace(repl_dict, regex=True).map(pd.eval).astype(int)

In [117]:
tweetDetailsDf.to_csv('./TrumpTweets.csv')

In [87]:
import re

date_time = re.sub(' EST$','',re.sub('(\d+)th|rd|st|nd','\g<1>',date.text))
dateObject = datetime.strptime(date_time,'%b %d %Y - %I:%M:%S %p')

'Jan 8 2021 - 10:44:28 AM'

In [88]:
from datetime import datetime


datetime.strptime(date_time,'%b %d %Y - %I:%M:%S %p')

datetime.datetime(2021, 1, 8, 10, 44, 28)

In [90]:
print(datetime.strptime('Jan 8 2021 - 10:44:28 PM','%b %d %Y - %I:%M:%S %p'))

2021-01-08 22:44:28


In [106]:
details = tweetElements[0].find_element(by=By.CLASS_NAME,value='icons___3WEom')
details.find_elements(by=By.TAG_NAME,value='span')

[<selenium.webdriver.remote.webelement.WebElement (session="b3cd2ab764daf38c93583e848825dca9", element="969fc058-80f6-49fb-88f2-a616d44f9fbd")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3cd2ab764daf38c93583e848825dca9", element="49f9ad58-33d8-43bd-952d-39ad3009709e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3cd2ab764daf38c93583e848825dca9", element="2541d951-54b6-488e-8543-41866a8f9e70")>]

In [102]:
tweet = tweetElements[0].find_element(by=By.CLASS_NAME,value='text___k4s4q.plain___1AdbJ').text
tweet

'To all of those who have asked, I will not be going to the Inauguration on January 20th.'

In [ ]:
import matplotlib as mpl
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
mpl.rcParams['figure.figsize']=(12.0,12.0)  
mpl.rcParams['font.size']=12            
mpl.rcParams['savefig.dpi']=100             
mpl.rcParams['figure.subplot.bottom']=.1 
stopwords = set(STOPWORDS)
wordcloud = WordCloud(
                          background_color='white',
                          max_words=500,
                          max_font_size=40, 
                          random_state=100
                         ).generate(str(tweets_df_clean.content))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show();